# Classification Evaluation Methods
<hr>

* **Accuracy:** Default metric for classification problems.
* **Precision:** Higher precision leads to less false positive.
* **Recall:** Higher recall leads to less false negative.
* **F1-score:** Combination of precision and recall.
* **Confusion matrix:** When comparing predictions with truth labels.

![Eval](../images/eval.JPG)